# Naive Bayes

O Naive Bayes é um classificador que se baseia em encontrar funções descrevendo a probabilidade de um dado pertencer a uma determinada classe.

Ele é baseado no teorema de Bayes que determina a probabilidade de um evento ocorrer dado que outro ocorreu, a partir da seguinte fórmula:

$$ P(A | B) = \frac{P(B | A) * P(A)}{P(B)} $$

Que diz a probabilidade do evento A ocorrer dado que o evento B ocorreu é a razão entre a probabilidade do evento B ocorrer dado que o evento A ocorreu vezes a probabilidade do evento A e a probabilidade do evento B.

Quando se vai resolver um problema usando este classificador existem 3 modelos disponíveis na biblioteca Python do Sklearn:

    - Gaussiano : é usado assumindo que os dados seguem uma distribuição normal.
    - Multinominal : é usado se os dados forem discretos.
    - Bernoulli : é usado quando os dados forem binários.    

Vamos usar neste exemplo o Gaussiano Naive Bayes ou GaussianNB. Ele funciona da seguinte forma: o dataset é dividido no total de classes que ele tiver e para cada classe vai ser calculado a média e o desvio padrão de cada atributo e montado uma distribuição gaussiana para cada classe. Assim quando chegar um novo dado é calculado a função de densidade de probabilidade gaussiana para cada classe, esse calculo mostra a probabilidade do novo valor de pertencer a determinada classe.

Para saber mais sobre o que foi usado neste notebook:

- [Explicação baseada neste site](https://blog.sicara.com/naive-bayes-classifier-sklearn-python-example-tips-42d100429e44)
- [GaussianNB](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html)
- [Modelos do scikit usando Naive Bayes](http://scikit-learn.org/stable/modules/naive_bayes.html)
- [train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)
- [np.where](https://docs.scipy.org/doc/numpy/reference/generated/numpy.where.html)
- [Problema do Titanic](https://www.kaggle.com/c/titanic)

OBS: Se quiser se divertir resolvendo problemas envolvendo aprendizagem de máquina pode entrar na plataforma Kaggle. Lá tem diversos problemas e até mesmo competições. ;)

# Problema

O problema que vamos tratar é um problema de classificação bem conhecido na área de aprendizagem de máquina chamado Titanic. O dataset reúne informações pessoais dos passageiros do Titanic e a partir disso devemos descobrir quem morre e quem sobrevive ao naufrágio. Como o dataset é muito grande e cheio de atributos, vamos diminuí-lo para ficar mais fácil de entender.

Vamos tentar fazer uma previsão de sobrevivência usando apenas as informações sobre sexo (M - masculino, F - feminino), porto de embarcação (C - Cherbourg, Q - Queenstown, S - Southampton) e classe do ticket (1 - classe alta, 2 - classe média, 3 - classe baixa).

#### Importando as bibliotecas que vamos usar

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

#### Importanto o dataset

In [2]:
df = pd.read_csv('train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#### Manipulando os dados

In [3]:
#retirando os atributos que não vamos usar 
df = df.drop(['PassengerId', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin'], axis=1)

#retirando as linhas com dados faltando, pois eles alteram muito o resultado quando usando o modelo do Naive Bayes
df = df.dropna(axis = 0, how = 'any')
df.head()

,Survived,Pclass,Sex,Embarked
0,0,3,male,S
1,1,1,female,C
2,1,3,female,S
3,1,1,female,S
4,0,3,male,S


#### Como o naive Bayes precisa de números para poder calcular as probabilidades, vamos converter as variáveis categóricas em númericas usando o np.where 

numpy.where(condition, x, y), se a condição for verdadeira retorna x, se não for false retorna y.

In [4]:
df["Sex"] = np.where(df["Sex"] == "male", 0, 1)
df["Embarked"] = np.where(df["Embarked"] == "S", 0,
                 np.where(df["Embarked"] == "C", 1,
                 np.where(df["Embarked"] == "Q", 2, 3)))

df.head()

,Survived,Pclass,Sex,Embarked
0,0,3,0,0
1,1,1,1,1
2,1,3,1,0
3,1,1,1,0
4,0,3,0,0


#### Separando os atributos da resposta

In [5]:
y = df["Survived"]
X = df.drop("Survived", axis=1)

#### Dividindo o conjunto de train e o conjunto de teste usando o train_test_split

train_test_split( * arrays , ** options ) -> ele divide matriz ou matrizes em subconjuntos aleatórios de treino e teste.

Options:
    - test_size -> representar a proporção do conjunto de dados a ser incluída no conjunto de teste
    - train_size -> representar a proporção do conjunto de dados a ser incluída na divisão de treino
    - stratify -> estratificar os dados

OBS: No test_size ou train_size pode-se colocar valores de float ou int, se for float tem que estar entre 0,0 e 1,0 pois ele representa a proporção do conjunto de dados a ser incluída na divisão. E se for int ele representa o número absoluto de amostras do conjunto formado. 

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, stratify = y)

#### Criando o modelo

In [7]:
Gnb = GaussianNB()

#### Treinando o modelo

In [8]:
Gnb.fit(X_train, y_train)

GaussianNB(priors=None)

#### Testando o modelo

In [9]:
Y_pred = Gnb.predict(X_test)

#### Calculando e imprimindo os resultados

In [11]:
total_teste = X_test.shape[0] #pegar apenas a quantidade de linhas do X_test
total_acerto = (y_test != Y_pred).sum()
desempenho = 100*(1-total_acerto/total_teste)

print("Total de casos de teste {} , com {} acertos, com um desempenho de {:05.2f}%".format(total_teste, 
                                                                                        total_acerto, 
                                                                                        desempenho))

Total de casos de teste 267 , com 61 acertos, com um desempenho de 77.15%
